In [2]:
from dhanhq import dhanhq, marketfeed
import talib
import numpy as np
import pandas as pd
import time
import datetime
import os
import logging

In [3]:
with open("token.txt", "r") as file:
    # Read the entire contents of the file
    token = file.read()
client_id = "1102249582"
access_token = token
dhan = dhanhq(client_id,access_token)

masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', nrows=1)
masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', usecols=masterlist.columns, low_memory=False)


logging.basicConfig(filename="logfile.txt", 
                    level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")


In [4]:
NIFTY_PREMIMUM = 26000

filtered_ce_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-11-04 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'CE') 
]
filtered_ce_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
153940,NSE,D,47680,OPTIDX,0,NIFTY-Nov2025-26000-CE,75.0,NIFTY 04 NOV 26000 CALL,2025-11-04 14:30:00,26000.0,CE,5.0,W,OP,NaN,NaN


In [5]:
filtered_pe_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-11-04 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'PE')
]

filtered_pe_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
153944,NSE,D,47691,OPTIDX,0,NIFTY-Nov2025-26000-PE,75.0,NIFTY 04 NOV 26000 PUT,2025-11-04 14:30:00,26000.0,PE,5.0,W,OP,NaN,NaN


In [9]:

SECURITY_ID = 47691

IS_ACTIVE = False
IS_OPEN_POSITION = False
quantity = 0
buy_price = 0     
stop_loss = 0

current_time = datetime.datetime.now()
current_second = current_time.second
time_str = current_time.strftime("%H:%M:%S")

current_date = datetime.datetime.now().strftime("%Y-%m-%d")

while IS_ACTIVE is False:
        current_second = datetime.datetime.now().second
        time.sleep(1)
        print("waiting to activate", current_second)
        if current_second == 1:
            IS_ACTIVE = True

while IS_ACTIVE is True and IS_OPEN_POSITION is False:
      current_second = datetime.datetime.now().second
      if current_second == 0:
            break
      print('Active',current_second)
      time.sleep(0.15)
      intraday = dhan.intraday_minute_data(str(SECURITY_ID), dhan.NSE_FNO, "OPTIDX", current_date, current_date, 1)

      print("intraday", intraday)
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

    

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]
      
      
     
      # if current_running_candle['close'] > previous_closed_candle['high']:
      #       print('Buy')
      #       length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
      #       risk                      = 1000
      #       quantity                  = round(round(risk / length_of_previous_candle) / 75) * 75
      #       print('quantity',quantity)
            
      #       if quantity > 200:
      #             quantity = 200

      #       buy_price = int(current_running_candle['close'])
      #       stop_loss = int(previous_closed_candle['low'])

      #       dhan.place_order(
      #       security_id=SECURITY_ID,
      #       exchange_segment=dhan.NSE_FNO,
      #       transaction_type=dhan.BUY,
      #       quantity=quantity,
      #       order_type=dhan.MARKET,
      #       product_type=dhan.INTRA,
      #       price=0)  

      #       logging.info(f"BUY at {buy_price}")

      #       IS_OPEN_POSITION = True
      #       break;
      print('Buy')
      length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
      risk                      = 1000
      quantity                  = 75 #round(round(risk / length_of_previous_candle) / 75) * 75
      print('quantity',quantity)
      
      if quantity > 200:
            quantity = 200

      buy_price = float(current_running_candle['close'])
      stop_loss = float(previous_closed_candle['low'])

      # dhan.place_order(
      # security_id=SECURITY_ID,
      # exchange_segment=dhan.NSE_FNO,
      # transaction_type=dhan.BUY,
      # quantity=quantity,
      # order_type=dhan.MARKET,
      # product_type=dhan.INTRA,
      # price=0)  

      logging.info(f"BUY at {buy_price}")
      print(f"BUY at {buy_price}")

      IS_OPEN_POSITION = True
      break




waiting to activate 42
waiting to activate 43
waiting to activate 44
waiting to activate 45
waiting to activate 46
waiting to activate 47
waiting to activate 48
waiting to activate 49
waiting to activate 50
waiting to activate 51
waiting to activate 52
waiting to activate 53
waiting to activate 54
waiting to activate 55
waiting to activate 56
waiting to activate 57
waiting to activate 58
waiting to activate 59
waiting to activate 0
waiting to activate 1
Active 2
intraday {'status': 'success', 'remarks': '', 'data': {'open': [150.0, 114.8, 115.65, 123.5, 128.35, 129.95, 126.5, 136.55, 134.5, 135.65, 127.6, 125.55, 120.15, 117.65, 120.05, 118.95, 116.15, 114.4, 118.95, 116.95, 121.3, 127.2, 128.9, 130.9, 134.35, 135.8, 135.35, 139.65, 136.4, 131.45, 134.4, 126.6, 122.0, 128.15, 137.85, 140.65, 134.0, 127.15, 128.2, 128.75, 125.15, 123.65, 121.25, 128.55, 129.85, 127.4, 121.7, 120.55, 117.7, 119.6, 121.4, 123.4, 124.35, 123.35], 'high': [153.05, 118.5, 124.7, 127.95, 131.1, 132.2, 135.0, 

In [ ]:

COST_TO_COST_SL = False
TRAILING_SL = False

while IS_OPEN_POSITION is True and quantity > 0:
      time.sleep(0.10)
      intraday = dhan.intraday_minute_data(str(SECURITY_ID), dhan.NSE_FNO, "OPTIDX", current_date, current_date, 1)
      print("waiting for sell")
      # intraday_df = pd.DataFrame(intraday['data']).tail(2)
      # temp_list = []

     
      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]

      # Stop loss
      if current_running_candle['close'] < stop_loss and COST_TO_COST_SL is False and TRAILING_SL is False:
            print("SELL")
            IS_OPEN_POSITION = False
          
      #      # Place an order for NSE Futures & Options
      #       dhan.place_order(security_id=SECURITY_ID,
      #       exchange_segment=dhan.NSE_FNO,
      #       transaction_type=dhan.SELL,
      #       quantity=quantity,
      #       order_type=dhan.MARKET,
      #       product_type=dhan.INTRA,
      #       price=0)  
            print(f"SL hit {current_running_candle['close']}")

      # Trailing SL
      if current_running_candle['close'] < previous_closed_candle['low'] and COST_TO_COST_SL is False and TRAILING_SL is True:
            print("SELL")
            IS_OPEN_POSITION = False
          
      #      # Place an order for NSE Futures & Options
      #       dhan.place_order(security_id=SECURITY_ID,
      #       exchange_segment=dhan.NSE_FNO,
      #       transaction_type=dhan.SELL,
      #       quantity=quantity,
      #       order_type=dhan.MARKET,
      #       product_type=dhan.INTRA,
      #       price=0)  
            print(f"Trailing SL hit {current_running_candle['close']}")

      # Shift sl to Cost to Cost
      if current_running_candle['close'] < buy_price +1 and COST_TO_COST_SL is True and TRAILING_SL is False:
            print("SELL")
            IS_OPEN_POSITION = False
          
      #      # Place an order for NSE Futures & Options
      #       dhan.place_order(security_id=SECURITY_ID,
      #       exchange_segment=dhan.NSE_FNO,
      #       transaction_type=dhan.SELL,
      #       quantity=quantity,
      #       order_type=dhan.MARKET,
      #       product_type=dhan.INTRA,
      #       price=0)  
            print(f"CTC SL hit {current_running_candle['close']}")

            